In [77]:
import numpy as np, cv2
import matplotlib.pyplot as plt

In [78]:
# load Model and Test
loaded = np.load("utils\\dataset.npz")
x = loaded['x']
y = loaded['y']

In [79]:
def remove_noise(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    se = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    bg = cv2.morphologyEx(img, cv2.MORPH_DILATE, se)
    
    out_gray = cv2.divide(img, bg, scale=255)
    #out_binary = cv2.threshold(out_gray, 0, 255, cv2.THRESH_OTSU)[1]
    
    return cv2.cvtColor(out_gray,cv2.COLOR_GRAY2RGB)
    

# Normalize and remove noise of the image
x_norm = []
for idx, data in enumerate(x):
    data = cv2.resize(data, (48, 48))
    noiseless = remove_noise(data)
    normalized = cv2.normalize(noiseless, noiseless, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    x_norm.append(normalized)
x_norm = np.asarray(x_norm)

In [80]:
# Lets do the cross validation
from sklearn.model_selection import StratifiedKFold as skf

skf_model = skf(n_splits=4, random_state=8, shuffle=True)
for train_idx, test_idx in skf_model.split(x, y):
    x_train, x_test = x_norm[train_idx], x_norm[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

<p> addition to balance the class : <br>
    Class 0 need 2000 <br>
    Class 1 need 2000 <br>
    Class 3 need 1300 <br>
    Class 4 need 1400 <br>
    Class 5 need 2377 + 300 <br>
</p>

In [81]:
from collections import Counter

counter = Counter(y_train)
for k, v in counter.items():
    per= v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=0, n=3000 (14.148%)
Class=1, n=3072 (14.487%)
Class=2, n=5411 (25.518%)
Class=3, n=3723 (17.557%)
Class=4, n=3622 (17.081%)
Class=5, n=2377 (11.210%)


In [82]:
def brightness_settings(img, brightness=255):    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        buf = cv2.addWeighted(img, alpha_b, img, 0, gamma_b)
    else:
        buf = img.copy()
        
    return buf

def contrast_settings(img, contrast=127):
    if contrast != 0:
        f = float(131 * (contrast + 127)) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        buf = cv2.addWeighted(img, alpha_c, img, 0, gamma_c)
        
    return buf

In [83]:
def imbalance_control(class_num, class_needed, value=0, augmentation='contrast', x=[], y=[]):
    augmented_x = []
    augmented_y = []
    start_idx = 0
    
    for idx, img_train in enumerate(x):
        if y[idx] == class_num:
            if augmentation == 'contrast':
                augmented_x.append(contrast_settings(img_train, value))
            if augmentation == 'brightness':
                augmented_x.append(brightness_settings(img_train, value))
            if augmentation == 'flip':
                augmented_x.append(cv2.flip(img_train, 2))
            augmented_y.append(y[idx])
            start_idx = start_idx + 1
        if start_idx == class_needed:
            break
            
    return augmented_x, augmented_y

In [84]:
# Lets Start the augmentation
augmented_class_x0, augmented_class_y0 = imbalance_control(0, 2000, 0, 'flip', x_train, y_train)
augmented_class_x1, augmented_class_y1 = imbalance_control(1, 2000, 0, 'flip', x_train, y_train)
augmented_class_x3, augmented_class_y3 = imbalance_control(3, 1300, 0, 'flip', x_train, y_train)
augmented_class_x4, augmented_class_y4 = imbalance_control(4, 1400, 0, 'flip', x_train, y_train)
augmented_class_x5, augmented_class_y5 = imbalance_control(5, 2377, 0, 'flip', x_train, y_train)
augmented_class_x5_b, augmented_class_y5_b = imbalance_control(5, 300, 50, 'brightness', x_train, y_train)

In [85]:
augmented_x = []
augmented_y = []

augmented_x.extend(augmented_class_x0)
augmented_x.extend(augmented_class_x1)
augmented_x.extend(augmented_class_x3)
augmented_x.extend(augmented_class_x4)
augmented_x.extend(augmented_class_x5)
augmented_x.extend(augmented_class_x5_b)

augmented_y.extend(augmented_class_y0)
augmented_y.extend(augmented_class_y1)
augmented_y.extend(augmented_class_y3)
augmented_y.extend(augmented_class_y4)
augmented_y.extend(augmented_class_y5)
augmented_y.extend(augmented_class_y5_b)

In [86]:
# Make it one array
x_train = x_train.tolist()
y_train = y_train.tolist()

In [87]:
x_train.extend(augmented_x)
y_train.extend(augmented_y)

In [88]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [89]:
counter = Counter(y_train)
for k, v in counter.items():
    per= v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=0, n=5000 (16.349%)
Class=1, n=5072 (16.585%)
Class=2, n=5411 (17.693%)
Class=3, n=5023 (16.425%)
Class=4, n=5022 (16.421%)
Class=5, n=5054 (16.526%)


In [90]:
np.savez("utils\\train.npz", x=x_train, y=y_train)
np.savez("utils\\test.npz", x=x_test, y=y_test)

In [91]:
x_train[0]

array([[[0.96862751, 0.96862751, 0.96862751],
        [0.94901967, 0.94901967, 0.94901967],
        [0.93725497, 0.93725497, 0.93725497],
        ...,
        [0.76862752, 0.76862752, 0.76862752],
        [0.82745105, 0.82745105, 0.82745105],
        [0.72156864, 0.72156864, 0.72156864]],

       [[0.94901967, 0.94901967, 0.94901967],
        [0.96862751, 0.96862751, 0.96862751],
        [0.93725497, 0.93725497, 0.93725497],
        ...,
        [0.72156864, 0.72156864, 0.72156864],
        [0.8705883 , 0.8705883 , 0.8705883 ],
        [0.80392164, 0.80392164, 0.80392164]],

       [[0.95686281, 0.95686281, 0.95686281],
        [0.95686281, 0.95686281, 0.95686281],
        [0.98039222, 0.98039222, 0.98039222],
        ...,
        [0.66666669, 0.66666669, 0.66666669],
        [0.72156864, 0.72156864, 0.72156864],
        [0.86274517, 0.86274517, 0.86274517]],

       ...,

       [[0.90588242, 0.90588242, 0.90588242],
        [0.91764712, 0.91764712, 0.91764712],
        [0.58431375, 0